In [3]:
# !pip install neo4j
# !pip install torch_geometric
# !pip install networkx
# !pip install torch

In [14]:
from neo4j import GraphDatabase
import networkx as nx
uri = "bolt://localhost:7687"
user = "neo4j"
password = "Swayam2003"


driver = GraphDatabase.driver(uri, auth=(user, password))

query1 = "MATCH (a:Author) RETURN COUNT(a) AS numAuthors"
query2 = "MATCH ()-[e:CO_AUTHORED]->() RETURN COUNT(e) AS numCoAuthorships;"
query3 = "MATCH ()-[e:CO_AUTHORED]->() MATCH (a:Author) With COUNT(a) AS numAuthors,COUNT(e) AS numCoAuthorships RETURN (2.0 * numCoAuthorships) / (numAuthors * (numAuthors - 1)) AS networkDensity;"
query4 = "MATCH (a:Author) MATCH (a)-[e:CO_AUTHORED]->() WITH a, COUNT(e) AS degree RETURN AVG(degree) AS averageDegree;"
query5 = "MATCH (a:Author)-[e:CO_AUTHORED]->() WITH a, COUNT(e) AS degree RETURN a, degree ORDER BY degree DESC LIMIT 10;"

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        records = [record for record in result]
        return records

print(run_query(query1))
print(run_query(query2))
print(run_query(query3))
print(run_query(query4))
print(run_query(query5))



driver.close()


[<Record numAuthors=347>]
[<Record numCoAuthorships=2519>]
[<Record networkDensity=2.2880886680120627e-06>]
[<Record averageDegree=8.178571428571427>]
[<Record a=<Node element_id='4:90d4a046-c6c5-4c79-987b-3b140a0d4b7c:15' labels=frozenset({'Author'}) properties={'Feature140': '0', 'Feature146': '0', 'Feature145': '0', 'Feature148': '0', 'Feature147': '0', 'Feature90': '0', 'Feature142': '0', 'Feature141': '0', 'Feature92': '0', 'Feature144': '0', 'Feature143': '0', 'Feature91': '0', 'Feature94': '0', 'Feature93': '0', 'Feature96': '0', 'Feature95': '0', 'Feature98': '0', 'Feature139': '0', 'Feature138': '0', 'Feature97': '0', 'Feature99': '0', 'Feature151': '0', 'Feature150': '0', 'Feature157': '0', 'Feature156': '0', 'Feature159': '0', 'Feature158': '0', 'Feature153': '0', 'Feature152': '0', 'Feature81': '0', 'Feature155': '0', 'Feature80': '0', 'Feature154': '0', 'Feature83': '0', 'Feature82': '0', 'Feature85': '0', 'Feature84': '0', 'Feature87': '0', 'Feature149': '0', 'Feature86':

GNN Testing


In [16]:
from neo4j import GraphDatabase
import networkx as nx
import torch
from torch_geometric.data import Data


uri = "bolt://localhost:7687"
user = "neo4j"
password = "Swayam2003"
driver = GraphDatabase.driver(uri, auth=(user, password))


def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        records = [record for record in result]
        return records


query = " MATCH (a:Author)-[:CO_AUTHORED]->(b:Author) RETURN a.author_id AS author1, b.author_id AS author2 "
records = run_query(query)


G = nx.Graph()
author_to_index = {}
index = 0
for record in records:
    author1 = record["author1"]
    author2 = record["author2"]
    if author1 not in author_to_index:
        author_to_index[author1] = index
        index += 1
    if author2 not in author_to_index:
        author_to_index[author2] = index
        index += 1
    G.add_edge(author_to_index[author1], author_to_index[author2])


author_features_query = "MATCH (a:Author) RETURN a.author_id AS author_id,a.Feature28 AS Feature28,a.Feature24 AS Feature24,        a.Feature27 AS Feature27,        a.Feature26 AS Feature26,        a.Feature29 AS Feature29,        a.Feature20 AS Feature20,        a.Feature23 AS Feature23,        a.Feature22 AS Feature22,        a.Feature25 AS Feature25,        a.Feature174 AS Feature174,        a.Feature177 AS Feature177,        a.Feature176 AS Feature176,        a.Feature21 AS Feature21,        a.Feature172 AS Feature172,        a.Feature179 AS Feature179,        a.Feature178 AS Feature178,        a.Feature175 AS Feature175,        a.Feature39 AS Feature39,        a.Feature171 AS Feature171,        a.Feature170 AS Feature170,        a.Feature173 AS Feature173,        a.Feature36 AS Feature36,        a.Feature35 AS Feature35,        a.Feature38 AS Feature38,        a.Feature37 AS Feature37,        a.Feature32 AS Feature32,        a.Feature31 AS Feature31,        a.Feature34 AS Feature34,        a.Feature33 AS Feature33,        a.Feature163 AS Feature163,        a.Feature166 AS Feature166,        a.Feature165 AS Feature165,        a.Feature30 AS Feature30,        a.Feature168 AS Feature168,        a.Feature167 AS Feature167,        a.Feature169 AS Feature169,        a.Feature164 AS Feature164,        a.Feature48 AS Feature48,        a.Feature160 AS Feature160,        a.Feature162 AS Feature162,        a.Feature161 AS Feature161,        a.Feature44 AS Feature44,        a.Feature47 AS Feature47,        a.Feature46 AS Feature46,        a.Feature49 AS Feature49,        a.Feature40 AS Feature40,        a.Feature43 AS Feature43,        a.Feature42 AS Feature42,        a.Feature45 AS Feature45,        a.Feature196 AS Feature196,        a.Feature199 AS Feature199,        a.Feature198 AS Feature198,        a.Feature41 AS Feature41,        a.Feature194 AS Feature194,        a.Feature191 AS Feature191,        a.Feature190 AS Feature190,        a.Feature197 AS Feature197,        a.Feature59 AS Feature59,        a.Feature193 AS Feature193,        a.Feature192 AS Feature192,        a.Feature195 AS Feature195,        a.Feature56 AS Feature56,        a.Feature55 AS Feature55,        a.Feature58 AS Feature58,        a.Feature57 AS Feature57,        a.Feature52 AS Feature52,        a.Feature51 AS Feature51,        a.Feature54 AS Feature54,        a.Feature53 AS Feature53,        a.Feature185 AS Feature185,        a.Feature188 AS Feature188,        a.Feature187 AS Feature187,        a.Feature50 AS Feature50,        a.Feature183 AS Feature183,        a.Feature180 AS Feature180,        a.Feature189 AS Feature189,        a.Feature186 AS Feature186,        a.Feature19 AS Feature19,        a.Feature182 AS Feature182,        a.Feature181 AS Feature181,        a.Feature184 AS Feature184,        a.Feature16 AS Feature16,        a.Feature15 AS Feature15,        a.Feature18 AS Feature18,        a.Feature17 AS Feature17,        a.Feature12 AS Feature12,        a.Feature11 AS Feature11,        a.Feature14 AS Feature14,        a.Feature13 AS Feature13,        a.Feature204 AS Feature204,        a.Feature207 AS Feature207,        a.Feature206 AS Feature206,        a.Feature10 AS Feature10,        a.Feature210 AS Feature210,        a.Feature209 AS Feature209,        a.Feature208 AS Feature208,        a.Feature205 AS Feature205,        a.Feature212 AS Feature212,        a.Feature211 AS Feature211,        a.Feature214 AS Feature214,        a.Feature213 AS Feature213,        a.Feature201 AS Feature201,        a.Feature200 AS Feature200,        a.Feature203 AS Feature203,        a.Feature202 AS Feature202,        a.Feature106 AS Feature106,        a.Feature105 AS Feature105,        a.Feature108 AS Feature108,        a.Feature107 AS Feature107,        a.Feature114 AS Feature114,        a.Feature111 AS Feature111,        a.Feature110 AS Feature110,        a.Feature109 AS Feature109,        a.Feature6 AS Feature6,        a.Feature113 AS Feature113,        a.Feature112 AS Feature112,        a.Feature115 AS Feature115,        a.Feature1 AS Feature1,        a.Feature9 AS Feature9,        a.Feature8 AS Feature8,        a.Feature7 AS Feature7,        a.Feature5 AS Feature5,        a.Feature4 AS Feature4,        a.Feature3 AS Feature3,        a.Feature2 AS Feature2,        a.Feature216 AS Feature216,        a.Feature215 AS Feature215,        a.Feature218 AS Feature218,        a.Feature217 AS Feature217,        a.Feature100 AS Feature100,        a.Feature221 AS Feature221,        a.Feature220 AS Feature220,        a.Feature219 AS Feature219,        a.Feature222 AS Feature222,        a.Feature104 AS Feature104,        a.Feature103 AS Feature103,        a.Feature224 AS Feature224,        a.Feature68 AS Feature68,        a.Feature102 AS Feature102,        a.Feature223 AS Feature223,        a.Feature101 AS Feature101,        a.Feature67 AS Feature67,        a.Feature66 AS Feature66,        a.Feature129 AS Feature129,        a.Feature69 AS Feature69,        a.Feature65 AS Feature65,        a.Feature128 AS Feature128,        a.Feature64 AS Feature64,        a.Feature127 AS Feature127,        a.Feature61 AS Feature61,        a.Feature60 AS Feature60,        a.Feature63 AS Feature63,        a.Feature62 AS Feature62,        a.Feature131 AS Feature131,        a.Feature130 AS Feature130,        a.Feature133 AS Feature133,        a.Feature132 AS Feature132,        a.Feature135 AS Feature135,        a.Feature134 AS Feature134,        a.Feature137 AS Feature137,        a.Feature136 AS Feature136,        a.Feature119 AS Feature119,        a.Feature77 AS Feature77,        a.Feature118 AS Feature118,        a.Feature79 AS Feature79,        a.Feature117 AS Feature117,        a.Feature75 AS Feature75,        a.Feature116 AS Feature116,        a.Feature78 AS Feature78,        a.Feature71 AS Feature71,        a.Feature74 AS Feature74,        a.Feature73 AS Feature73,        a.Feature76 AS Feature76,        a.Feature70 AS Feature70,        a.Feature122 AS Feature122,        a.Feature121 AS Feature121,        a.Feature72 AS Feature72,        a.Feature123 AS Feature123,        a.Feature126 AS Feature126,        a.Feature125 AS Feature125,        a.Feature120 AS Feature120,        a.Feature149 AS Feature149,        a.Feature89 AS Feature89,        a.Feature88 AS Feature88,        a.Feature124 AS Feature124,        a.Feature85 AS Feature85,        a.Feature84 AS Feature84,        a.Feature87 AS Feature87,        a.Feature86 AS Feature86,        a.Feature80 AS Feature80,        a.Feature154 AS Feature154,        a.Feature83 AS Feature83,        a.Feature82 AS Feature82,        a.Feature153 AS Feature153,        a.Feature152 AS Feature152,        a.Feature81 AS Feature81,        a.Feature155 AS Feature155,        a.Feature157 AS Feature157,        a.Feature156 AS Feature156,        a.Feature159 AS Feature159,        a.Feature158 AS Feature158,        a.Feature138 AS Feature138,        a.Feature99 AS Feature99,        a.Feature151 AS Feature151,        a.Feature150 AS Feature150,        a.Feature95 AS Feature95,        a.Feature98 AS Feature98,        a.Feature139 AS Feature139,        a.Feature97 AS Feature97,        a.Feature143 AS Feature143,        a.Feature94 AS Feature94,        a.Feature93 AS Feature93,        a.Feature96 AS Feature96,        a.Feature141 AS Feature141,        a.Feature92 AS Feature92,        a.Feature144 AS Feature144,        a.Feature91 AS Feature91,        a.Feature148 AS Feature148,        a.Feature147 AS Feature147,        a.Feature90 AS Feature90,        a.Feature142 AS Feature142,        a.Feature140 AS Feature140,        a.Feature146 AS Feature146,        a.Feature145 AS Feature145  ORDER BY a.author_id"
author_features_records = run_query(author_features_query)


author_features_dict = {}
for record in author_features_records:
    author_id = record["author_id"]
    features = [int(record[f"Feature{i}"]) for i in range(1, 225)]  # Assuming features are named Feature1, Feature2, ..., Feature225
    author_features_dict[author_id] = features

edge_index = torch.tensor(list(G.edges), dtype=torch.int).t().contiguous()

sorted_author_ids = sorted(author_to_index, key=author_to_index.get)

x = torch.tensor([author_features_dict[author_id] for author_id in sorted_author_ids], dtype=torch.float)

num_classes = 2 
y = torch.randint(0, num_classes, (x.size(0),), dtype=torch.float)

data = Data(x=x,y=y, edge_index=edge_index)

print(data)

driver.close()

import pickle

# Save the data using Pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)
with open('author_to_index.pickle', 'wb') as f:
    pickle.dump(author_to_index, f)


Data(x=[333, 224], edge_index=[2, 2519], y=[333])


In [4]:
# Access the first edge using G[0]
first_edge = G[0]

# Loop through the keys (target nodes) and their attributes
for target_node, attributes in first_edge.items():
    print("Target Node:", target_node)
    print("Attributes:", attributes)

Target Node: 1
Attributes: {}
Target Node: 31
Attributes: {}
Target Node: 8
Attributes: {}
Target Node: 140
Attributes: {}
Target Node: 2
Attributes: {}
Target Node: 24
Attributes: {}
Target Node: 17
Attributes: {}
Target Node: 70
Attributes: {}
Target Node: 106
Attributes: {}
Target Node: 16
Attributes: {}
Target Node: 178
Attributes: {}
Target Node: 22
Attributes: {}
Target Node: 41
Attributes: {}
Target Node: 18
Attributes: {}
Target Node: 68
Attributes: {}
Target Node: 110
Attributes: {}
Target Node: 236
Attributes: {}
Target Node: 56
Attributes: {}
Target Node: 182
Attributes: {}
Target Node: 249
Attributes: {}


In [5]:
print(data.edge_index)

tensor([[  0,   0,   0,  ..., 310, 319, 329],
        [  1,  31,   8,  ..., 311, 320, 330]], dtype=torch.int32)


In [6]:
print((author_to_index))

{'authorID_766cb_53c75_3baed_ac5dc_78259': 0, 'authorID_9a049_b03f6_fc40b_fcf2f_13632': 1, 'authorID_5f9c4_ab08c_ac745_7e911_1a30e': 2, 'authorID_1c6c0_bb2c7_ecdc3_be8e1_34f79': 3, 'authorID_c76b4_05781_134be_1dab7_fe45a': 4, 'authorID_b7a56_873cd_771f2_c446d_369b6': 5, 'authorID_624b6_0c58c_9d8bf_b6ff1_886c2': 6, 'authorID_7f0a2_2117f_8fe01_72cf9_209ff': 7, 'authorID_27bad_c983d_f1780_b60c2_b3fa9': 8, 'authorID_4c970_004b0_678d4_39f17_7e77d': 9, 'authorID_74806_4be03_a08df_81e31_bd6f9': 10, 'authorID_d2f48_3672c_0239f_6d7dd_3c9ec': 11, 'authorID_8bd9c_0d453_53375_7387e_d019c': 12, 'authorID_aae02_12936_2d611_717b6_c00ad': 13, 'authorID_377ad_eb4cd_4096a_dc7ca_64b53': 14, 'authorID_3fdba_35f04_dc8c4_62986_c992b': 15, 'authorID_d6e5a_20b30_f8721_6b2c7_58f5e': 16, 'authorID_49d18_0ecf5_61328_19571_bf39d': 17, 'authorID_6f4b6_61212_5fb3a_0daec_d2799': 18, 'authorID_81b8a_03f97_e8787_c53fe_1a86b': 19, 'authorID_44c80_31cb0_36a73_50d8b_9b860': 20, 'authorID_5ef6f_df325_13aa7_cd11f_72bec': 2

In [7]:
data.y

tensor([0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.,
        0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0.,
        1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
        1., 0., 1., 1., 0., 1., 0., 1., 

In [8]:
print(data.x)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

class GNNModel(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GNNModel, self).__init__()
        self.conv = GNNConv(num_features, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return x

class GNNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GNNConv, self).__init__(aggr="add")  # "add" aggregation
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        return x_j

    def update(self, aggr_out):
        return aggr_out

torch.manual_seed(1234)

num_features = data.num_node_features
hidden_channels = 64
num_classes = 1

model = GNNModel(num_features, hidden_channels, num_classes)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

data.train_mask = data.test_mask = None
data.train_mask, data.test_mask = train_test_split(range(data.num_nodes), test_size=0.2)
print(data.y)
data.x = data.x.float()
print(data.x)

model.train()
for epoch in range(100):
    total_loss = 0
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

print("Training finished!")

torch.save(model.state_dict(), 'model.pt')


tensor([0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.,
        0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0.,
        1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
        1., 0., 1., 1., 0., 1., 0., 1., 

c:\bal\balkan\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([266])) that is different to the input size (torch.Size([266, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [11]:
def index_to_author( value):
    for key, val in author_to_index.items():
        if val == value:
            return key

In [12]:
import numpy as np

loaded_model = GNNModel(num_features, hidden_channels, num_classes)
loaded_model.load_state_dict(torch.load('model.pt'))


def get_closest_coauthors(model, data, author_index, k=5):
    original_tensor = data.x[author_index] 
    original_tensor = original_tensor.unsqueeze(0)  
    new_tensor = torch.zeros((332, original_tensor.shape[1]))
    temp = torch.cat((original_tensor, new_tensor), dim=0)

    model.eval()
    with torch.no_grad():
        similarity_scores = model(temp, data.edge_index)

    closest_coauthor_indices = np.argsort(similarity_scores.squeeze().numpy())[:k]
    
    closest_coauthor_ids = [index_to_author(index) for index in closest_coauthor_indices]
    closest_coauthor_scores = similarity_scores.squeeze().numpy()[closest_coauthor_indices]

    return closest_coauthor_indices, closest_coauthor_ids, closest_coauthor_scores

author_id = 'authorID_4a44d_c1536_4204a_80fe8_0e903'
author_index = author_to_index[author_id]
indices, author_ids, likeliness_scores = get_closest_coauthors(loaded_model, data, author_index)

for i, (author_index, author_id, score) in enumerate(zip(indices, author_ids, likeliness_scores)):
    print(f"Rank {i+1}: Author ID: {author_id},Likeliness: {score:.4f}")


Rank 1: Author ID: authorID_766cb_53c75_3baed_ac5dc_78259,Likeliness: 0.2421
Rank 2: Author ID: authorID_9a049_b03f6_fc40b_fcf2f_13632,Likeliness: 0.2562
Rank 3: Author ID: authorID_5f9c4_ab08c_ac745_7e911_1a30e,Likeliness: 0.2778
Rank 4: Author ID: authorID_69f59_c273b_6e669_ac32a_6dd5e,Likeliness: 0.2864
Rank 5: Author ID: authorID_c0509_a487a_18b00_3ba05_e5054,Likeliness: 0.2864


Brute Force

In [13]:
from neo4j import GraphDatabase
import networkx as nx
uri = "bolt://localhost:7687"
user = "neo4j"
password = "Swayam2003"

driver = GraphDatabase.driver(uri, auth=(user, password))
author = 'authorID_9a049_b03f6_fc40b_fcf2f_13632'

query1 = "MATCH (source:Author)-[:CO_AUTHORED]->(target:Author) WHERE source.author_id = '"+author+"' RETURN target.author_id ORDER BY source.author_id LIMIT 5"

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        records = [record for record in result]
        return records

print(run_query(query1))

driver.close()


[<Record target.author_id='authorID_84a50_92e4a_5b6fe_968fd_523fb'>, <Record target.author_id='authorID_8b940_be7fb_78aaa_6b656_7dd7a'>, <Record target.author_id='authorID_2c7d5_490e6_05083_6f8f2_f0d49'>, <Record target.author_id='authorID_1be00_34108_2e25c_4e251_ca671'>, <Record target.author_id='authorID_d874e_4e4a5_df211_73b0f_83e31'>]
